### Import statements

In [46]:
import sys
sys.path.append('../venv/lib/python3.7/site-packages')
import warnings
import logging
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor

import mlflow.sklearn
from mlflow import log_param
from mlflow.tracking import MlflowClient

# <center> Training and logging your model on a local mlflow server </center>

In [2]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore")
np.random.seed(40)

/home/guillaume/code/GGIML/Wine-MLFlow/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# Read the wine-quality csv file from the URL
csv_url = \
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e)


In [4]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [5]:
X = data.drop('quality', axis=1)
y = data.quality

In [6]:
# Split the data into training and test sets. (0.75, 0.25) split.
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [42]:
alpha = 0.0001
l1_ratio = 0.5
normalize = False

In [43]:
def create_experiment(experiment_name) -> int:
    """
    Creates an mlflow experiment
    :param experiment_name: str. The name of the experiment to be set in MLFlow
    :return: the id of the experiment created if it doesn't exist, or the id of the existing experiment if it is already
    :return: the id of the experiment created if it doesn't exist, or the id of the existing experiment if it is already
    created
    """
    client = MlflowClient()
    experiments = client.list_experiments()
    experiment_names = set(map(lambda e: e.name, experiments))
    if experiment_name in experiment_names:
        print(f'Experiment {experiment_name} already created.')
        return list(filter(lambda e: e.name == experiment_name, experiments))[0].experiment_id
    else:
        return mlflow.create_experiment(name=experiment_name)

In [44]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [45]:
experiment_id = create_experiment(experiment_name='vivadata')
with mlflow.start_run(experiment_id=experiment_id, nested=False, run_name='ElasticNet'):
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    
    #Train a machine learning model of your choice with scikit learn
    elastic = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, normalize=normalize)
    elastic.fit(X_train, y_train)
    
    #Make prediction
    y_pred = elastic.predict(X_test)
    
    #Look at some metrics like the rmse
    rmse, mae, r2 = eval_metrics(y_test, y_pred)
    
    #Now your can log the params and metrics of your model
    mlflow.log_param('alpha', alpha)
    mlflow.log_param('l1_ratio', l1_ratio)
    mlflow.log_param('normalize', normalize)
    mlflow.log_metric('rmse', rmse)
    mlflow.log_metric('mae', mae)
    mlflow.log_metric('r2', r2)
    
    #Log your scikit learn model
    mlflow.sklearn.log_model(elastic, "ElasticNet")

Experiment vivadata already created.


In [60]:
learning_rate = 0.5
n_estimators = 1000
np.random.seed(40)

In [61]:
experiment_id = create_experiment(experiment_name='vivadata')
with mlflow.start_run(experiment_id=experiment_id, nested=False, run_name='GradientBoosting'):
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    
    #Train a machine learning model of your choice with scikit learn
    gbr = GradientBoostingRegressor(learning_rate=learning_rate, n_estimators=n_estimators)
    gbr.fit(X_train, y_train)
    
    #Make prediction
    y_pred = gbr.predict(X_test)
    
    #Look at some metrics like the rmse
    rmse, mae, r2 = eval_metrics(y_test, y_pred)
    
    #Now your can log the params and metrics of your model
    mlflow.log_param('learning_rate', learning_rate)
    mlflow.log_param('n_estimators', n_estimators)
    mlflow.log_metric('rmse', rmse)
    mlflow.log_metric('mae', mae)
    mlflow.log_metric('r2', r2)
    
    #Log your scikit learn model
    mlflow.sklearn.log_model(gbr, "GradientBoostingRegressor")

Experiment vivadata already created.
